# build hypothesis and cost

## H(x)= Wx + b 


In [48]:
import tensorflow as tf
import numpy as np

In [49]:
x_data = [1, 2, 3, 4, 5]
y_data = [1, 2, 3, 4, 5]
W = tf.Variable(2.9)
b = tf.Variable(0.5) 
#초기값은 임의의 값
hypothesis= W * x_data + b
cost = tf.reduce_mean(tf.square(hypothesis - y_data))

## Gradient descent 
* 경사를 하강하면서 최소값을 찾음 
* with tf.GradientTape() as tape: 이후에 나오는 것들을 tape에 기록
* A.assign_sub(B) 뜻?
    A = A - B
    A -= B
* 임의의 매우작은 값 learning rate 만큼 반영해서 업데이트해나감

In [50]:
learning_rate = 0.01

for i in range(100+1):
    with tf.GradientTape() as tape:
        hypothesis= W * x_data + b    
        cost = tf.reduce_mean(tf.square(hypothesis - y_data))
#경사도 구하기 (각 변수 W, b에 대해 cost 값을 미분한 기울기 값)
    W_grad, b_grad = tape.gradient(cost, [W, b]) 

#W, b 값을 업데이트 
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    if i % 10 == 0:
        print("{:5}|{:10.4f}|{:10.4}|{:10.6f}".format(i, W.numpy(), b.numpy(), cost))


    0|    2.4520|     0.376| 45.660004
   10|    1.1036|  0.003398|  0.206336
   20|    1.0128|  -0.02091|  0.001026
   30|    1.0065|  -0.02184|  0.000093
   40|    1.0059|  -0.02123|  0.000083
   50|    1.0057|  -0.02053|  0.000077
   60|    1.0055|  -0.01984|  0.000072
   70|    1.0053|  -0.01918|  0.000067
   80|    1.0051|  -0.01854|  0.000063
   90|    1.0050|  -0.01793|  0.000059
  100|    1.0048|  -0.01733|  0.000055


In [34]:
# 값을 대입해서 예측해보기
print(W * 5 + b)
print(W * 2.5 + b)

tf.Tensor(5.00667, shape=(), dtype=float32)
tf.Tensor(2.4946702, shape=(), dtype=float32)


# How to minimize cost
* 간략화된 모델로 우선 진행 H(x) = Wx   
* for feed_W in np.linspace(-3, 5, num=15):
 -3~5까지 15개 구간으로 나눈 값을 feed_W가 가짐 

In [35]:
# pure Python 으로 Cost function 만들기
X = np.array([1,2,3])
Y = np.array([1,2,3])

def cost_func(W, X, Y):
    c = 0
    for i in range(len(X)):
        c += (W * X[i] - Y[i]) ** 2
    return c / len(X)

for feed_W in np.linspace(-3, 5, num=15):
    curr_cost = cost_func(feed_W, X, Y)
    print("{:6.3f} | {:10.5f}".format(feed_W, curr_cost))

-3.000 |   74.66667
-2.429 |   54.85714
-1.857 |   38.09524
-1.286 |   24.38095
-0.714 |   13.71429
-0.143 |    6.09524
 0.429 |    1.52381
 1.000 |    0.00000
 1.571 |    1.52381
 2.143 |    6.09524
 2.714 |   13.71429
 3.286 |   24.38095
 3.857 |   38.09524
 4.429 |   54.85714
 5.000 |   74.66667


In [36]:
# Tensorflow로 Cost Function 만들기
def cost_func2(W, X, Y):
    hypothesis2 = X * W
    return tf.reduce_mean(tf.square(hypothesis2 - Y))

W_values = np.linspace(-3, 5, num=15)
cost_values = []

for feed_W in W_values:
    curr_cost = cost_func(feed_W, X, Y)
    cost_values.append(curr_cost)
    print("{:6.3f} | {:10.5f}".format(feed_W, curr_cost))

-3.000 |   74.66667
-2.429 |   54.85714
-1.857 |   38.09524
-1.286 |   24.38095
-0.714 |   13.71429
-0.143 |    6.09524
 0.429 |    1.52381
 1.000 |    0.00000
 1.571 |    1.52381
 2.143 |    6.09524
 2.714 |   13.71429
 3.286 |   24.38095
 3.857 |   38.09524
 4.429 |   54.85714
 5.000 |   74.66667


In [47]:
# gradient 함수 쓰지 않고 직접 만들어서 구해보기

tf.random.set_seed(0) # 다음에 다시 돌렸을때도 똑같이 나올 수 있도록 고정
X = [1, 2, 3, 4]
Y = [1, 2, 3, 4]
W= tf.Variable(tf.random.normal([1], -100, 100)) #normal 분포 중 임의의 값 1개


for step in range(300):
    hypothesis3 = W * X
    cost3 = tf.reduce_mean(tf.square(hypothesis3 - Y))
    alpha = 0.01
    gradient = tf.reduce_mean(tf.multiply(tf.multiply(W, X) - Y, X))
    descent = W - tf.multiply(alpha, gradient)
    W.assign(descent)
    
    if step % 10 ==0:
        print('{:5} | {:10.4f} | {:10.6f}'.format(step, cost3.numpy(), W.numpy()[0]))

    0 | 18829.7812 |  47.348293
   10 |  3959.8613 |  22.254509
   20 |   832.7499 |  10.746943
   30 |   175.1255 |   5.469776
   40 |    36.8285 |   3.049760
   50 |     7.7449 |   1.939984
   60 |     1.6287 |   1.431060
   70 |     0.3425 |   1.197676
   80 |     0.0720 |   1.090651
   90 |     0.0151 |   1.041571
  100 |     0.0032 |   1.019064
  110 |     0.0007 |   1.008742
  120 |     0.0001 |   1.004009
  130 |     0.0000 |   1.001839
  140 |     0.0000 |   1.000843
  150 |     0.0000 |   1.000387
  160 |     0.0000 |   1.000178
  170 |     0.0000 |   1.000081
  180 |     0.0000 |   1.000037
  190 |     0.0000 |   1.000017
  200 |     0.0000 |   1.000008
  210 |     0.0000 |   1.000004
  220 |     0.0000 |   1.000002
  230 |     0.0000 |   1.000001
  240 |     0.0000 |   1.000001
  250 |     0.0000 |   1.000001
  260 |     0.0000 |   1.000001
  270 |     0.0000 |   1.000001
  280 |     0.0000 |   1.000001
  290 |     0.0000 |   1.000001
